In [3]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-fs-prebuilt' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'nvidia*'


In [4]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

--2021-12-20 09:27:59--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.195.19.142
Connecting to developer.nvidia.com (developer.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 [following]
--2021-12-20 09:27:59--  https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?ARBXcXsJ9CCuBxyvDjshZjvtIIiyIFQ5PmJqQpFmQWb8ZNgK3I0CAst1NZKqlTR-UeB3Le5oXqa4GQ5VVTRRfgoqpElplREnDxMaO3vk_L54_

In [5]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Wed_Apr_11_23:16:29_CDT_2018
Cuda compilation tools, release 9.2, V9.2.88


In [6]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-d998wwm2
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-d998wwm2
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4305 sha256=4a7bee3dea937f5d021f853e9cdd17c26ab68ff86536229d649dfedd506858d9
  Stored in directory: /tmp/pip-ephem-wheel-cache-tnagddz1/wheels/c5/2b/c0/87008e795a14bbcdfc7c846a00d06981916331eb980b6c8bdf
Successfully built NVCCPlugin


In [7]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [8]:
%%cu
#include <stdio.h>
#include <stdlib.h>
__global__ void add(int *a, int *b, int *c) {
*c = *a + *b;
}
int main() {
int a, b, c;
// host copies of variables a, b & c
int *d_a, *d_b, *d_c;
// device copies of variables a, b & c
int size = sizeof(int);
// Allocate space for device copies of a, b, c
cudaMalloc((void **)&d_a, size);
cudaMalloc((void **)&d_b, size);
cudaMalloc((void **)&d_c, size);
// Setup input values  
c = 0;
a = 3;
b = 5;
// Copy inputs to device
cudaMemcpy(d_a, &a, size, cudaMemcpyHostToDevice);
  cudaMemcpy(d_b, &b, size, cudaMemcpyHostToDevice);
// Launch add() kernel on GPU
add<<<1,1>>>(d_a, d_b, d_c);
// Copy result back to host
cudaError err = cudaMemcpy(&c, d_c, size, cudaMemcpyDeviceToHost);
  if(err!=cudaSuccess) {
      printf("CUDA error copying to Host: %s\n", cudaGetErrorString(err));
  }
printf("result is %d\n",c);
// Cleanup
cudaFree(d_a);
cudaFree(d_b);
cudaFree(d_c);
return 0;
}

result is 8



In [9]:
%%cu
#include<stdio.h>
#include<cuda.h>

__global__ void arradd(int *x,int *y, int *z)    //kernel definition
{
  int id=blockIdx.x; 
/* blockIdx.x gives the respective block id which starts from 0 */
  z[id]=x[id]+y[id];
}

int main()
{
    int a[6] =  {1,6,4,6,7,2};
    int b[6] =  {2,3,7,1,-7,-2};
    int c[6];
    int *d,*e,*f;
    int i;
    

/* cudaMalloc() allocates memory from Global memory on GPU */
    cudaMalloc((void **)&d,6*sizeof(int));
    cudaMalloc((void **)&e,6*sizeof(int));
    cudaMalloc((void **)&f,6*sizeof(int));

/* cudaMemcpy() copies the contents from destination to source. Here destination is GPU(d,e) and source is CPU(a,b) */
 cudaMemcpy(d,a,6*sizeof(int),cudaMemcpyHostToDevice);   
 cudaMemcpy(e,b,6*sizeof(int),cudaMemcpyHostToDevice);
 
/* call to kernel. Here 6 is number of blocks, 1 is the number of threads per block and d,e,f are the arguments */ 
arradd<<<6,1>>>(d,e,f); 

/* Here we are copying content from GPU(Device) to CPU(Host) */
 cudaMemcpy(c,f,6*sizeof(int),cudaMemcpyDeviceToHost);
    
printf("\nSum of two arrays:\n ");
    for(i=0;i<6;i++)
    {
        printf("%d\t",c[i]);
    }

/* Free the memory allocated to pointers d,e,f */
    cudaFree(d);
    cudaFree(e);
    cudaFree(f);

    return 0;
}


Sum of two arrays:
 3	9	11	7	0	0	


# ***HPC A1: (Min,Max,Mean, Std. Deviation)***

In [11]:
%%cu
#include<iostream>
#include<math.h>

#define n 8

using namespace std;

__global__ void minimum(int *input) {
    int tid = threadIdx.x;
    int step_size = 1;
    int number_of_threads = blockDim.x;
    
    printf("No of threads = %d\n", number_of_threads);
    
    while(number_of_threads>0) {
        if(tid < number_of_threads) {
            int first = tid*step_size*2;
            int second = first + step_size;
            if(input[second] < input[first])
              input[first] = input[second];
        }
        step_size <<= 1;
        number_of_threads >>= 1;
    }
}

__global__ void maximum(int *input) {
    int tid = threadIdx.x;
    int step_size = 1;
    int number_of_threads = blockDim.x;
    
    while(number_of_threads>0) {
        if(tid < number_of_threads) {
            int first = tid*step_size*2;
            int second = first + step_size;
            if(input[second] > input[first])
              input[first] = input[second];
        }
        step_size <<= 1;
        number_of_threads >>= 1;
    }
}

__global__ void sum(int *input) {
    const int tid = threadIdx.x;
    int step_size = 1;
    int number_of_threads = blockDim.x;
    
    while(number_of_threads > 0) {
        if(tid < number_of_threads) {
            int first = tid * step_size * 2;
            int second = first + step_size;
            
            input[first] += input[second];
        }
        step_size <<= 1;
        number_of_threads >>= 1;
       
    }
}

__global__ void mean_diff_sq(float *input, float mean) {
    input[threadIdx.x] -= mean;
    input[threadIdx.x] *= input[threadIdx.x];
}

__global__ void sum_floats(float *input) {
    int tid = threadIdx.x;
    int step_size = 1;
    int number_of_threads = blockDim.x;
    
    while(number_of_threads > 0) {
        if(tid < number_of_threads) {
            int first = tid * step_size * 2;
            int second = first + step_size;
            
            input[first] += input[second];
        }
        step_size <<= 1;
        number_of_threads >>= 1;
       
    }
}

void copy_int_to_float(float *dest, int *src, int size){
    for(int i=0; i<size; i++)
        dest[i] = float(src[i]);
}

void random_ints(int *input, int size) {
    for(int i=0; i<size; i++)  {
        input[i] = rand()%100;
        cout<<input[i]<<"  ";   
    }
    cout<<endl;

}

int main() {
    //int n=8;
    int size = n*sizeof(int); //calculate no. of bytes for array
        
    int *arr;
    int *arr_d, result;
   
    arr = (int *)malloc(size);
    random_ints(arr, n);
    
    cudaMalloc((void **)&arr_d, size);
    
    //MIN
    cudaMemcpy(arr_d, arr, size, cudaMemcpyHostToDevice);
    
    minimum<<<1,n/2>>>(arr_d);
    
    cudaMemcpy(&result, arr_d, sizeof(int), cudaMemcpyDeviceToHost);
    
    cout<<"The minimum element is "<<result<<endl;
      
       
    //MAX
    cudaMemcpy(arr_d, arr, size, cudaMemcpyHostToDevice);
    
    maximum<<<1,n/2>>>(arr_d);
    
    cudaMemcpy(&result, arr_d, sizeof(int), cudaMemcpyDeviceToHost);
    
    cout<<"The maximum element is "<<result<<endl;
    
    //SUM
    cudaMemcpy(arr_d, arr, size, cudaMemcpyHostToDevice);
    
    sum<<<1,n/2>>>(arr_d);
    
    cudaMemcpy(&result, arr_d, sizeof(int), cudaMemcpyDeviceToHost);
    
    cout<<"The sum is "<<result<<endl;
    
    //AVERAGE
    
    float mean = float(result)/n;
    cout<<"The mean is "<<mean<<endl;
    
    //STANDARD DEVIATION
    float *arr_float;
    float *arr_std, stdValue;
    
    arr_float = (float *)malloc(n*sizeof(float));
    cudaMalloc((void **)&arr_std, n*sizeof(float));
    
    copy_int_to_float(arr_float, arr, n);
    
    cudaMemcpy(arr_std, arr_float, n*sizeof(float), cudaMemcpyHostToDevice);
    
    mean_diff_sq <<<1,n>>>(arr_std, mean);
    sum_floats<<<1,n/2>>>(arr_std);
    
    cudaMemcpy(&stdValue, arr_std, sizeof(float), cudaMemcpyDeviceToHost);
    
    
    stdValue = stdValue / n;
    cout<<"The variance is "<<stdValue<<endl;
    stdValue = sqrt(stdValue);
    
    cout<<"The standard deviation is "<<stdValue<<endl;
    
    cudaFree(arr_d);
           
    return 0;
}


83  86  77  15  93  35  86  92  
No of threads = 4
No of threads = 4
No of threads = 4
No of threads = 4
The minimum element is 15
The maximum element is 93
The sum is 567
The mean is 70.875
The variance is 748.359
The standard deviation is 27.3562



# ***HPC: A2 Parallel Computing (Matrix-Matrix Multiplication)***

In [12]:
%%cu
#include<iostream>

using namespace std;

__global__
void matrixMultiplication(int *a, int *b, int *c, int m, int n, int k)
{
    int row = blockIdx.y*blockDim.y + threadIdx.y;
    int col = blockIdx.x*blockDim.x + threadIdx.x;
    int sum=0;
   
    if(col<k && row<m) {
      for(int j=0;j<n;j++)
      {
          sum += a[row*n+j] * b[j*k+col];
      }
      c[k*row+col]=sum;
    }
    
}

void init_result(int *a, int m, int k) {
    for(int i=0; i<m; i++) {
      for(int j=0; j<k; j++) {
        a[i*k + j] = 0;
      }
    }
}

void init_matrix(int *a, int n, int m) {
    for(int i=0; i<n; i++) {
      for(int j=0; j<m; j++) {
        a[i*m + j] = rand()%10 + 1;
      }
    }
}

void print_matrix(int *a, int n, int m) {
    for(int i=0; i<n; i++) {
      for(int j=0; j<m; j++) {
        cout<<"  "<<a[i*m + j];
      }
      cout<<endl;
    }
    cout<<endl;
}

int main()
{
    
    int *a,*b,*c;
    int *a_dev,*b_dev,*c_dev;
    int m=5, n=4, k=3;
    
    a = new int[m*n];
    b = new int[n*k];
    c = new int[m*k];
    
    init_matrix(a, m, n);
    init_matrix(b, n ,k);
    init_result(c, m, k);
    
    cout<<"Initial matrix : "<<endl;
    
    print_matrix(a, m, n);
    print_matrix(b, n, k);
    print_matrix(c, m, k);
    
    cudaMalloc(&a_dev, sizeof(int)*m*n);
    cudaMalloc(&b_dev, sizeof(int)*n*k);
    cudaMalloc(&c_dev, sizeof(int)*m*k);
       
    cudaMemcpy(a_dev, a, sizeof(int)*m*n, cudaMemcpyHostToDevice);
    cudaMemcpy(b_dev, b, sizeof(int)*n*k, cudaMemcpyHostToDevice);
    
    dim3 dimGrid(1,1);
    dim3 dimBlock(16,16);
    
    matrixMultiplication<<<dimGrid, dimBlock>>>(a_dev,b_dev,c_dev, m, n, k);
     
    cudaMemcpy(c, c_dev, sizeof(int)*m*k, cudaMemcpyDeviceToHost);
    
    cout<<"Result : "<<endl;
    print_matrix(c, m, k);
    
    cudaFree(a_dev);
    cudaFree(b_dev);
    cudaFree(c_dev);
    
    delete[] a;
    delete[] b;
    delete[] c;
    
    return 0;
}


Initial matrix : 
  4  7  8  6
  4  6  7  3
  10  2  3  8
  1  10  4  7
  1  7  3  7

  2  9  8
  10  3  1
  3  4  8
  6  10  3

  0  0  0
  0  0  0
  0  0  0
  0  0  0
  0  0  0

Result : 
  138  149  121
  107  112  103
  97  188  130
  156  125  71
  123  112  60




# ***HPC: A2 Parallel Computing (Vector-Matrix Multiplication)***

In [13]:
%%cu
#include<iostream>

using namespace std;

__global__
void matrixVector(int *vec, int *mat, int *result, int n, int m)
{
    int tid = blockIdx.x*blockDim.x + threadIdx.x;
    int sum=0;
    
    if(tid <= n) {
        for(int i=0; i<n; i++) {
            sum += vec[i]*mat[(i*m) + tid];
        }
        result[tid] = sum;
    }
}

void init_array(int *a, int n) {
    for(int i=0; i<n; i++)
      a[i] = rand()%n + 1;
}

void init_matrix(int *a, int n, int m) {
    for(int i=0; i<n; i++) {
        for(int j=0; j<m; j++) {
            a[i*m + j] = rand()%n + 1;
        }
    }
}

void print_array(int *a, int n) {
    for(int i=0; i<n; i++) {
        cout<<"  "<<a[i];
    }
    cout<<endl;
}

void print_matrix(int *a, int n, int m) {
    for(int i=0; i<n; i++) {
        for(int j=0; j<m; j++)
          cout<<"  "<<a[i*m + j];
        cout<<endl;
    }
}

int main() {
    int *a, *b, *c;
    int *a_dev, *b_dev, *c_dev;
    
    int n = 3;
    int m = 4;
    
    a = new int[n];
    b = new int[n*m];
    c = new int[m];
    
    init_array(a, n);
    init_matrix(b, n, m);
        
    cout<<"Initial array : "<<endl;
    print_array(a, n);
    cout<<"Initial matrix : "<<endl;
    print_matrix(b, n, m);
    cout<<"Initial resultant array : "<<endl;
    print_array(c, m);
    cout<<endl;
    
    cudaMalloc(&a_dev, sizeof(int)*n);
    cudaMalloc(&b_dev, sizeof(int)*n*m);
    cudaMalloc(&c_dev, sizeof(int)*m);
    
    cudaMemcpy(a_dev, a, sizeof(int)*n, cudaMemcpyHostToDevice);
    cudaMemcpy(b_dev, b, sizeof(int)*n*m, cudaMemcpyHostToDevice);
    
    matrixVector<<<m/256+1, 256>>>(a_dev, b_dev, c_dev, n, m);
    
    cudaMemcpy(c, c_dev, sizeof(int)*m, cudaMemcpyDeviceToHost);
    
    cout<<"Results : "<<endl;
    print_array(c, m);
    
    cudaFree(a_dev);
    cudaFree(b_dev);
    cudaFree(c_dev);
    
    delete[] a;
    delete[] b;
    delete[] c;
    
    return 0;
}

Initial array : 
  2  2  1
Initial matrix : 
  2  3  2  2
  1  1  2  3
  2  3  2  3
Initial resultant array : 
  0  0  0  0

Results : 
  8  11  10  13



# ***HPC: A2 Parallel Computing (Vector-Vector Addition)***

In [14]:
%%cu
#include<iostream>
#include<cstdlib>
using namespace std;

__global__ void vectorAdd(int *a, int *b, int *result, int n) {
    int tid = blockIdx.x*blockDim.x + threadIdx.x;
    if(tid <= n) {
        result[tid] = a[tid] + b[tid];
    }
}

void print_array(int *a, int N) {
    for(int i=0; i<N; i++) {
        cout<<"  "<<a[i];
    }
    cout<<endl;
}

void init_array(int *a, int N) {
    for(int i=0; i<N; i++) {
        a[i] = rand()%10 + 1;
    }
}

int main() {
    int *a, *b, *c;
    int *a_dev, *b_dev, *c_dev;
    int n = 8;           //24
    
    a = (int*)malloc(n * sizeof(n));
    b = (int*)malloc(n * sizeof(n));
    c = (int*)malloc(n * sizeof(n));

    int size = n * sizeof(int);
    
    cudaMalloc(&a_dev, size);
    cudaMalloc(&b_dev, size);
    cudaMalloc(&c_dev, size);
    
    init_array(a, n);
    init_array(b, n);
    
    print_array(a, n);
    print_array(b, n);
        
    //cudaEvent_t start, end;
    //cudaEventCreate(&start);
    //cudaEventCreate(&end);
    
    cudaMemcpy(a_dev, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(b_dev, b, size, cudaMemcpyHostToDevice);
    
    //int threads = 1024;
    //int blocks = (n+threads-1)/threads;
    
    //cudaEventRecord(start);
    
    //vectorAdd<<<blocks,threads>>>(a_dev, b_dev, c_dev, n);
    vectorAdd<<<1,1024>>>(a_dev, b_dev, c_dev, n);
    
    //cudaEventRecord(end);
    
    //cudaDeviceSynchronize();
       
    //float time = 0.0;
    //cudaEventElapsedTime(&time, start, end);
    
    cudaMemcpy(c, c_dev, size, cudaMemcpyDeviceToHost);
    
    cout<<"Results : "<<endl;
    print_array(c, n);
 
    //cout<<"Time elapsed : "<<time<<endl;
        
    cudaFree(a_dev);
    cudaFree(b_dev);
    cudaFree(c_dev);
        
    return 0;
}

  4  7  8  6  4  6  7  3
  10  2  3  8  1  10  4  7
Results : 
  14  9  11  14  5  16  11  10



# ***HPC A3: Parallel Bubble Sort***

In [29]:
code = """
#include<omp.h>
#include<iostream>
using namespace std;

void swap(int *num1, int *num2) {
    int temp = *num1;
    *num1 = *num2;
    *num2 = temp;
}

int main() {
    int n = 10;
    int a[n];
    
    omp_set_num_threads(2);
    
    for(int i=0; i<n; i++) {
        a[i] = rand()% 100;
    }
    
    for(int i=0; i<n; i++) 
        cout<<"  "<<a[i];
    cout<<endl;
    
    int i=0, j=0;
    int first=0;
    double start, end;
    
    start = omp_get_wtime();
    for(i=0; i<n-1; i++) {
        first = i%2;
        #pragma omp parallel for
        for(j=first; j<n-1; j++) {
            if(a[j] > a[j+1])
              swap(&a[j], &a[j+1]);
        }
    }
    
    end = omp_get_wtime();
    cout<<"Result(parallel) : "<<endl;
    for(i=0; i<n; i++)
      cout<<"  "<<a[i];
    cout<<endl;
    
    cout<<"Time parallel = "<<(end-start)<<endl;
    
    return 0;
    
}
"""

In [26]:
text_file = open("code.cpp", "w")
text_file.write(code)
text_file.close()

In [27]:
!g++ -fopenmp code.cpp

In [28]:
!./a.out

  83  86  77  15  93  35  86  92  49  21
Result(parallel) : 
  15  21  35  49  77  83  86  86  92  93
Time parallel = 9.6007e-05


# ***HPC A3: Merge Sort Parallel***

In [58]:
code = """
#include<iostream>
#include<omp.h>

using namespace std;

void printArray(int *arr, int size) {
    for(int i=0; i<size; i++) {
         cout<<arr[i]<<"  ";
    }
    cout<<endl;
}

void merge(int* arr, int start, int mid, int end) {
    int len = (end - start) + 1;
    int temp[len];
    int cur = 0;
    
    int i = start;
    int j = mid + 1;
    while(i <= mid && j <= end){
        if(arr[i] < arr[j]) {
            temp[cur] = arr[i];
            cur++;
            i++;
        }
        else {
            temp[cur] = arr[j];
            cur++;
            j++;
        }
    }
    if(i <= mid) {
        while(i <= mid) {
            temp[cur] = arr[i];
            i++;
            cur++;
        }
    }
    
    else if(j <= end) {
        while(j <= end) {
            temp[cur] = arr[j];
            j++;
            cur++;
        }
    }
    
    cur = 0;
    for(i=start; i<=end; i++) {
        arr[i] = temp[cur];
        cur++;
    }

}
//merge sort normal
void mergeSortNormal(int *array, int begin, int end)
{
    if (begin >= end)
        return; // Returns recursively
  
    auto mid = begin + (end - begin) / 2;
    mergeSortNormal(array, begin, mid);
    mergeSortNormal(array, mid + 1, end);
    merge(array, begin, mid, end);
}

//merge sort parallel
void mergeSort(int *arr, int start, int end) {
    if(start < end) {
        int mid = (start+end) / 2;
        
        #pragma omp parallel sections
        {
            #pragma omp section
            mergeSort(arr, start, mid);
            
            #pragma omp section
            mergeSort(arr, mid+1, end);
        }
        
        merge(arr, start, mid, end);
    
    }
}

int main(int argc, char *argv[]) {
    int size = 100;
    int a[size];
    
    double start, end;
    
    omp_set_num_threads(2);
    
    for(int i=0; i<size; i++) {
        a[i] = rand()% 100;
    }
    
    //int a[]= {7,33,5,5,23,111,75,34,77,121,120};
    
    for(int i=0; i<size; i++) 
        cout<<"  "<<a[i];
    cout<<endl;
    start = omp_get_wtime();
    
    mergeSort(a, 0, size-1);
    
    printArray(a, size);
    
    end = omp_get_wtime();
    
    cout<<"Time parallel = "<<(end-start)<<endl;

    //for normal merge sort
    int b[size];
    for(int i=0; i<size; i++) {
        b[i] = rand()% 100;
    }

    double start2 = omp_get_wtime();
    
    mergeSortNormal(b, 0, size-1);
    
    printArray(b, size);
    
    double end2 = omp_get_wtime();
    
    cout<<"Time Normal Merge Sort = "<<(end2-start2)<<endl;


    
    return 0;
}

"""

In [59]:
text_file = open("merge11.cpp", "w")
text_file.write(code)
text_file.close()

In [60]:
!g++ -fopenmp merge11.cpp

In [61]:
!./a.out

  83  86  77  15  93  35  86  92  49  21  62  27  90  59  63  26  40  26  72  36  11  68  67  29  82  30  62  23  67  35  29  2  22  58  69  67  93  56  11  42  29  73  21  19  84  37  98  24  15  70  13  26  91  80  56  73  62  70  96  81  5  25  84  27  36  5  46  29  13  57  24  95  82  45  14  67  34  64  43  50  87  8  76  78  88  84  3  51  54  99  32  60  76  68  39  12  26  86  94  39
2  3  5  5  8  11  11  12  13  13  14  15  15  19  21  21  22  23  24  24  25  26  26  26  26  27  27  29  29  29  29  30  32  34  35  35  36  36  37  39  39  40  42  43  45  46  49  50  51  54  56  56  57  58  59  60  62  62  62  63  64  67  67  67  67  68  68  69  70  70  72  73  73  76  76  77  78  80  81  82  82  83  84  84  84  86  86  86  87  88  90  91  92  93  93  94  95  96  98  99  
Time parallel = 0.000239181
0  1  1  2  3  6  7  8  9  11  14  15  17  17  18  18  19  19  19  21  21  23  24  24  27  27  28  28  28  28  28  29  29  29  29  31  32  32  34  34  35  36  37  37  40  40  41  4

# ***HPC 4 : MPI (Binary Search)***

In [62]:
code = """
#include<mpi.h>
#include<stdio.h>

#define n 12

#define key 55

int a[] = {1,2,3,4,7,9,13,24,55,56,67,88};

int a2[20];

int binarySearch(int *array, int start, int end, int value) {
    int mid;
    
    while(start <= end) {
        mid = (start + end) / 2;
        if(array[mid] == value) 
            return mid;
        else if(array[mid] > value)
            end = mid - 1;
        else
            start = mid + 1;
    }
    return -1;
}

int main(int argc, char* argv[]) {
    int pid, np, elements_per_process, n_elements_received;
    
    MPI_Status status;
    
    MPI_Init(&argc, &argv);
    
    MPI_Comm_rank(MPI_COMM_WORLD, &pid);
    MPI_Comm_size(MPI_COMM_WORLD, &np);
    
    if(pid == 0) {
        int index, i;
        
        if(np > 1) {
            for(i=1; i<np-1; i++) {
                
                index = i * elements_per_process;
                //element count
                MPI_Send(&elements_per_process, 1, MPI_INT, i, 0, MPI_COMM_WORLD);

                MPI_Send(&a[index], elements_per_process, MPI_INT, i, 0, MPI_COMM_WORLD);
            
            }
            
            index = i* elements_per_process;
            
            int elements_left = n - index;
            
            MPI_Send(&elements_left, 1, MPI_INT, i, 0, MPI_COMM_WORLD);
            
            MPI_Send(&a[index], elements_left, MPI_INT, i, 0, MPI_COMM_WORLD);
        }
        
        int position = binarySearch(a, 0, elements_per_process-1, key);
        
        if(position != -1)
          printf("Found at: %d", position);
        
        int temp;
        
        for(i=1; i<np; i++) {
            MPI_Recv(&temp, 1, MPI_INT, MPI_ANY_SOURCE, 0, MPI_COMM_WORLD, &status);
            int sender = status.MPI_SOURCE;
            
            if(temp != -1)
                printf("Found at: %d by %d", (sender*elements_per_process)+temp, sender);
        }
    }
    
    else {
        MPI_Recv(&n_elements_received, 1, MPI_INT, 0, 0, MPI_COMM_WORLD, &status);
        
        MPI_Recv(&a2, n_elements_received, MPI_INT, 0, 0, MPI_COMM_WORLD, &status);
    
        int position = binarySearch(a2, 0, n_elements_received-1, key);
        
        MPI_Send(&position, 1, MPI_INT, 0, 0, MPI_COMM_WORLD);
    }
    
    MPI_Finalize();
    
    return 0;
}
"""


In [63]:
text_file = open("mpiBinary.c", "w");
text_file.write(code);
text_file.close();

In [64]:
!mpiCC mpiBinary.c

In [65]:
!mpirun --allow-run-as-root -np 4 ./a.out

Found at: 8 by 3

# ***HPC A4 : (BFS)***

In [68]:
code = """
#include<iostream>
#include<omp.h>

using namespace std;
int q[100];
int visited[7];
int local_q;

void bfs(int adj_matrix[7][7], int first, int last, int q[], int n_nodes) {
    if(first==last)
      return;
    
    int cur_node = q[first++];
    cout<<"  "<<cur_node;
    
    omp_set_num_threads(3);
    
    #pragma omp parallel for shared(visited)
    for(int i=0; i<n_nodes; i++) {
        if(adj_matrix[cur_node][i] == 1 && visited[i] == 0){
            q[last++] = i;
            visited[i] = 1;
        }
    }
    
    bfs(adj_matrix, first, last, q, n_nodes);
}

int main() {
    int first = -1;
    int last = 0;
    int n_nodes = 7;
    
    for(int i=0; i<n_nodes; i++) {
        visited[i] = 0;
    }
    
    int adj_matrix[7][7] = {
      {0, 1, 1, 0, 0, 0, 0},
      {1, 0, 1, 1, 0, 0, 0},
      {1, 1, 0, 0, 1, 0, 0},
      {0, 1, 0, 0, 1, 0, 0},
      {0, 0, 1, 1, 0, 1, 0},
      {0, 0, 0, 0, 1, 0, 1},
      {0, 0, 0, 0, 0, 1, 0}
    };
    
    int start_node = 3;
    q[last++] = start_node;
    first++;
    visited[start_node] = 1;
 
    bfs(adj_matrix, first, last, q, n_nodes);
    
    return 0;
    
}
"""

In [67]:
text_file = open("code.cpp", "w")
text_file.write(code)
text_file.close()

In [69]:
!g++ -fopenmp code.cpp

In [70]:
!./a.out

  3  1  4  0  2  5  6